In [1]:
import cdsapi
import xarray as xr
import glob
import csv

In [2]:
c = cdsapi.Client()

In [3]:
!pwd

/home/sonjastndl/s3/ClimateData


In [ ]:

c.retrieve(
    'derived-near-surface-meteorological-variables',
    {
        'version': '2.1',
        'format': 'tgz',
        'variable': 'near_surface_air_temperature',
        'year': [
            '2012', '2013', '2014',
            '2015', '2016', '2017',
            '2018', '2019',
        ],
        'month': [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
        ],
        'reference_dataset': 'cru',
    },
    '/home/sonjastndl/s3/ClimateData/climatedata/download.tar.gz')

In [ ]:
!tar -xvzf /home/sonjastndl/s3/ClimateData/climatedata/download.tar.gz 

In [6]:
file_pattern = "/home/sonjastndl/s3/ClimateData/climatedata/Tair_WFDE5_CRU_*.nc"
files = glob.glob(file_pattern)
# Open multiple datasets and concatenate them along the time dimension
ds = xr.open_mfdataset(files, combine='by_coords')

print(ds)

<xarray.Dataset>
Dimensions:  (time: 70128, lon: 720, lat: 360)
Coordinates:
  * time     (time) datetime64[ns] 2012-01-01 ... 2019-12-31T23:00:00
  * lon      (lon) float64 -179.8 -179.2 -178.8 -178.2 ... 178.8 179.2 179.8
  * lat      (lat) float64 -89.75 -89.25 -88.75 -88.25 ... 88.75 89.25 89.75
Data variables:
    Tair     (time, lat, lon) float32 dask.array<chunksize=(744, 360, 720), meta=np.ndarray>
Attributes:
    title:        WATCH Forcing Data methodology applied to ERA5 data
    institution:  Copernicus Climate Change Service
    contact:      http://copernicus-support.ecmwf.int
    comment:      Methodology implementation for ERA5 and dataset production ...
    Conventions:  CF-1.7
    summary:      ERA5 data regridded to half degree regular lat-lon; Genuine...
    reference:    Cucchi et al., 2020, Earth Syst. Sci. Data, 12(3), 2097–212...
    licence:      The dataset is distributed under the Licence to Use Coperni...


In [13]:
output_csv_file="/home/sonjastndl/s3/ClimateData/TESTMARIA_5.csv"

In [8]:
with open("/home/sonjastndl/s3/LGA/uc3-drosophola-genetics/projects/LandscapeGenomicsPipeline/EOXHUB_TEST3/dest_v2.samps_3May2024.csv", newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in spamreader:
        try:
            day=int(row[10])
        except ValueError:
            continue
        month=int(row[12])
        year=row[13]
        sampleId=row[0]
        lat=row[3]
        lon=row[4]
        day2="{0:0=2d}".format(day)
        month2="{0:0=2d}".format(month)
        date=year+"-"+month2+"-"+day2
        #print(date)
        with open (output_csv_file, "a") as o:
            spamwriter = csv.writer(o, delimiter=',')
            TG_location = ds.Tair.sel(lon=lon,lat=lat, method='nearest')
            try:
                Subset_time_and_loation= TG_location.sel(time=str(date))
                WW=Subset_time_and_loation.to_dataframe()
                mtemp= WW.Tair.mean()
                #print(date)
                #print(mtemp)
                o.write(sampleId + "," + str(mtemp)+"\n")
            except KeyError:
                o.write(sampleId + "," + "NA" +"\n")
            #o.write("test" +sampleId + str(mtemp)+"\n")

In [14]:
with open("/home/sonjastndl/s3/LGA/uc3-drosophola-genetics/projects/LandscapeGenomicsPipeline/EOXHUB_TEST3/dest_v2.samps_3May2024.csv", newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    with open(output_csv_file, "a") as o:
        spamwriter = csv.writer(o, delimiter=',')
        o.write("sampleId,NSAT" +"\n")
        # Iterate over rows in the input CSV file
        for row in spamreader:
            try:
                day = int(row[10])
            except ValueError:
                continue
            month = int(row[12])
            year = row[13]
            sampleId = row[0]
            lat = row[3]
            lon = row[4]
            day2 = "{0:0=2d}".format(day)
            month2 = "{0:0=2d}".format(month)
            date = year + "-" + month2 + "-" + day2 
            # Write header once if not already written
            TG_location = ds.Tair.sel(lon=lon, lat=lat, method='nearest')
            try:
                Subset_time_and_loation= TG_location.sel(time=str(date))
                WW=Subset_time_and_loation.to_dataframe()
                mtemp= WW.Tair.mean()
                #print(date)
                #print(mtemp)
                o.write(sampleId + "," + str(mtemp)+"\n")
            except KeyError:
                o.write(sampleId + "," + "NA" +"\n")

In [ ]:
##if you want e.g. have NSAT 50 days before sampling
#get average of weeks before sampling
#import datetime
#u = datetime.datetime.strptime(date,"%Y-%m-%d")
#d = datetime.timedelta(days=50)
#t = u - d
#print(t)

##spanwidth of temperatures for that day/week/moth whatever
#abs(WW.Tair.min()-WW.Tair.max())
